In [ ]:
# --- COLAB SETUP START ---
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("🔧 Running in Google Colab - Installing dependencies...")
    
    # Install backend package
    !pip install -q git+https://github.com/yourusername/gemini-fullstack-langgraph-quickstart.git#subdirectory=backend
    
    # Or if running from a local copy
    # !pip install -q -e ./backend
    
    print("✅ Dependencies installed!")
else:
    print("✅ Running locally")
# --- COLAB SETUP END ---

In [ ]:
# Universal Setup for Backend Environment
import sys
import os
import subprocess
from pathlib import Path

def setup_environment():
    """Setup the environment by installing necessary dependencies and setting paths."""
    # Get the backend directory. If we are in 'backend', it is cwd.
    backend_dir = Path.cwd()
    if backend_dir.name != 'backend':
        # Search for backend
        if (backend_dir / 'backend').exists():
             backend_dir = backend_dir / 'backend'
        elif (backend_dir.parent / 'backend').exists():
             backend_dir = backend_dir.parent / 'backend'
    
    # Add src to path if it exists (for 'from agent import ...' style)
    src_dir = backend_dir / 'src'
    if src_dir.exists():
        if str(src_dir) not in sys.path:
            sys.path.append(str(src_dir))
            print(f"  [OK] Added {src_dir} to sys.path")
    
    if str(backend_dir) not in sys.path:
        sys.path.append(str(backend_dir))
        
    # Verify backend/agent can be imported
    try:
        import agent
        print("  [OK] Agent module found and imported.")
    except ImportError:
        print("  [!] Agent module not found. Installing dependencies...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", str(backend_dir)])
        print("  [OK] Backend installed in editable mode.")

setup_environment()

In [ ]:
# --- MODEL CONFIGURATION ---
# @title Select Gemini Model
# @markdown Choose the Gemini model to use. Only Gemini 2.5 models are currently accessible via the API.

MODEL_STRATEGY = "Gemini 2.5 Flash (Recommended)" # @param ["Gemini 2.5 Flash (Recommended)", "Gemini 2.5 Flash-Lite (Fastest)", "Gemini 2.5 Pro (Best Quality)"]

import os

# Map selection to model ID
# Note: Gemini 1.5 and 2.0 models are deprecated/not accessible via this API
if MODEL_STRATEGY == "Gemini 2.5 Flash (Recommended)":
    SELECTED_MODEL = "gemma-3-27b-it"
elif MODEL_STRATEGY == "Gemini 2.5 Flash-Lite (Fastest)":
    SELECTED_MODEL = "gemma-3-27b-it-lite"
elif MODEL_STRATEGY == "Gemini 2.5 Pro (Best Quality)":
    SELECTED_MODEL = "gemma-3-27b-it"
else:
    # Default fallback
    SELECTED_MODEL = "gemma-3-27b-it"

print(f"Selected Model: {SELECTED_MODEL}")
print(f"Strategy: {MODEL_STRATEGY}")

# Set Environment Variables to override defaults
os.environ["QUERY_GENERATOR_MODEL"] = SELECTED_MODEL
os.environ["REFLECTION_MODEL"] = SELECTED_MODEL
os.environ["ANSWER_MODEL"] = SELECTED_MODEL
os.environ["TOOLS_MODEL"] = SELECTED_MODEL

In [ ]:
# --- MODEL VERIFICATION (Optional) ---
# @title Verify Model Configuration
# @markdown Run this cell to verify that your API key is configured correctly and the selected model is available.

import os

# Check if API key is set
if "GEMINI_API_KEY" not in os.environ:
    print("⚠️  GEMINI_API_KEY not found in environment variables!")
    print("   Please set it before proceeding:")
    print("   export GEMINI_API_KEY='your-api-key-here'")
else:
    try:
        from google import genai
        
        # Initialize the client
        client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])
        
        # Test the selected model
        print(f"🧪 Testing model: {SELECTED_MODEL}")
        response = client.models.generate_content(
            model=SELECTED_MODEL,
            contents="Explain how AI works in a few words"
        )
        
        print(f"  [OK] Model verification successful!")
        print(f"   Model: {SELECTED_MODEL}")
        print(f"   Response: {response.text[:100]}...")
        
    except ImportError:
        print("  [!] google-genai package not installed!")
        print("   Installing now...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "google-genai"])
        print("  [OK] Installed! Please re-run this cell.")
        
    except Exception as e:
        print(f"  [X] Model verification failed: {e}")
        print(f"   This could mean:")
        print(f"   - Invalid API key")
        print(f"   - Model '{SELECTED_MODEL}' not available in your region")
        print(f"   - Quota/billing issues (for experimental models)")
        print(f"   - Network connectivity issues")

# Gemini Fullstack LangGraph - Colab Setup

This notebook demonstrates how to install and run the Gemini Fullstack LangGraph backend agent in Google Colab.

**Note:** You will need a Google Gemini API Key.

In [ ]:
# 1. Clone the repository
!rm -rf gemini-fullstack-langgraph-quickstart
!git clone https://github.com/MasumRab/gemini-fullstack-langgraph-quickstart
%cd gemini-fullstack-langgraph-quickstart/backend

In [ ]:
# 2. Prepare Environment (Resolving Conflicts)

import sys

# Google Colab comes with pre-installed packages (like TensorFlow and older Google AI libs)
# that require older versions of Protobuf. The Agent requires a newer Protobuf.
# We uninstall the conflicting packages to ensure a clean installation.
print("Uninstalling conflicting pre-installed packages...")
!pip uninstall -y google-ai-generativelanguage tensorflow grpcio-status

# Check Python version
print(f"Python Version: {sys.version}")

# Pre-install PyTorch Nightly if Python 3.13+ is detected (Compatibility Fix)
if sys.version_info >= (3, 13):
    print("Detected Python 3.13+. Installing PyTorch Nightly for compatibility...")
    !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

In [ ]:
# 3. Install Backend
!pip install .

In [ ]:
# 4. Set API Key
import os
from google.colab import userdata

try:
    # Try to get from Colab Secrets (recommended)
    os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
except:
    # Fallback to manual input
    print("Please enter your Gemini API Key:")
    os.environ["GEMINI_API_KEY"] = input()

In [ ]:
# 5. Run the Agent CLI Example

# We can verify the installation by running a simple CLI query
!python ../examples/cli_research.py "What are the main advantages of LangGraph?"